In [1]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Usable Code

In [2]:
!pwd

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [3]:
files = sorted([f for f in os.listdir('csv') if '.csv' in f])

In [4]:
files

['07-08.csv',
 '08-09.csv',
 '09-10.csv',
 '10-11.csv',
 '11-12.csv',
 '12-13.csv',
 '13-14.csv',
 '14-15.csv',
 '15-16.csv',
 '16-17.csv',
 '17-18.csv',
 '18-19.csv',
 '19-20.csv']

In [5]:
cd csv

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv


In [52]:
def clean_odds(file):
    '''
    cleans csv file and returns dataframe object that contains the over/under lines (target)
    '''
    df = pd.read_csv(file)
    df = df.iloc[:,:13] #some files contained unnamed extra rows
    
    #makes a list of game_id's equivalent to how many games were played that season
    mylist = []
    for i in range(1, int(df.shape[0]/2 + 1)):
        mylist.append(i)
        mylist.append(i)
    df['game_id'] = mylist #maps game_id's to games (spread across 2 rows)
    
    df = df.merge(df, on='game_id', suffixes=(None,'_2')) #gets games to be in one row (creates 4/game)
    df = df[1::4] #gets correct mapping
    df.reset_index(drop=True, inplace=True)
    
    #make dates include year so we can join with another dataset later
    df['Date'] = df['Date'].astype('str')
    dates = []
    for date in df['Date']:
        if int(date) > 1000:
            date = '20'+file[0][:2]+date #GET FILENAMES AND USE HERE
            dates.append(date)
        else:
            date = '20'+file[0][3:5]+'0'+date #GET FILENAMES AND USE HERE
            dates.append(date)
    df['Date'] = dates
    
    #take out pick-em's and replace with zero for later transformation
    df['Open'].replace('pk','0',inplace=True)
    df['Close'].replace('pk','0',inplace=True)
    df['Open_2'].replace('pk','0',inplace=True)
    df['Close_2'].replace('pk','0',inplace=True)
    df.replace('197.5u10','197.5',inplace=True)
    #map strings as floats for comparison
    df['Open'] = df['Open'].astype('float')
    df['Open_2'] = df['Open_2'].astype('float')
    df['Close'] = df['Close'].astype('float')
    df['Close_2'] = df['Close_2'].astype('float')
    #get correct over/under line at opening and closing of sportsbook
    df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
    df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
    
    #rename columns to correct home/visitor 
    df['Visitor'] = df['Team']
    df['Home'] = df['Team_2']
    
    #drop unneeded info
    df = df[['Date','Home','Visitor','O/U_open','O/U_close']]
    return df

In [53]:
for file in files:
    clean_odds(file)

ValueError: invalid literal for int() with base 10: '1016.0'

## Scratch Work

In [6]:
df = pd.read_csv('07-08.csv')

In [7]:
df.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95
1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5
2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5
3,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3
4,1030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    2632 non-null   int64 
 1   Rot     2632 non-null   int64 
 2   VH      2632 non-null   object
 3   Team    2632 non-null   object
 4   1st     2632 non-null   int64 
 5   2nd     2632 non-null   int64 
 6   3rd     2632 non-null   int64 
 7   4th     2632 non-null   int64 
 8   Final   2632 non-null   int64 
 9   Open    2632 non-null   object
 10  Close   2632 non-null   object
 11  ML      2632 non-null   object
 12  2H      2632 non-null   object
dtypes: int64(7), object(6)
memory usage: 267.4+ KB


In [9]:
df = df.iloc[:,:13]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2632 entries, 0 to 2631
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    2632 non-null   int64 
 1   Rot     2632 non-null   int64 
 2   VH      2632 non-null   object
 3   Team    2632 non-null   object
 4   1st     2632 non-null   int64 
 5   2nd     2632 non-null   int64 
 6   3rd     2632 non-null   int64 
 7   4th     2632 non-null   int64 
 8   Final   2632 non-null   int64 
 9   Open    2632 non-null   object
 10  Close   2632 non-null   object
 11  ML      2632 non-null   object
 12  2H      2632 non-null   object
dtypes: int64(7), object(6)
memory usage: 267.4+ KB


Just trying on 2 games

In [11]:
df1 = df.iloc[:4,:]

In [12]:
df1

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95
1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5
2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5
3,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3


In [13]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 300)

In [14]:
df.shape[0]

2632

In [15]:
mylist = []
for i in range(1, int(df.shape[0]/2 + 1)):
    mylist.append(i)
    mylist.append(i)
print(mylist[0],mylist[-1])

1 1316


In [16]:
len(range(1,int(df.shape[0]/2 + 1)))

1316

In [17]:
df['game_id'] = mylist

In [18]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,1
1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,1
2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2
3,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3,2
4,1030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627,612,508,H,LALakers,35,23,15,18,91,8,7.5,-355,96.5,1314
2628,615,509,V,Boston,22,30,18,28,98,190.5,193,255,97.5,1315
2629,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4,1315
2630,617,511,V,LALakers,20,15,25,32,92,192,190,170,5,1316


In [19]:
df = df.merge(df, on='game_id', suffixes=(None,'_2'))

In [20]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,1,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95
1,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5
2,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,1,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95
3,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5
4,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5259,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4,1315,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4
5260,617,511,V,LALakers,20,15,25,32,92,192,190,170,5,1316,617,511,V,LALakers,20,15,25,32,92,192,190,170,5
5261,617,511,V,LALakers,20,15,25,32,92,192,190,170,5,1316,617,512,H,Boston,24,34,31,42,131,3.5,4.5,-200,95
5262,617,512,H,Boston,24,34,31,42,131,3.5,4.5,-200,95,1316,617,511,V,LALakers,20,15,25,32,92,192,190,170,5


In [21]:
df = df[1::4]
df.reset_index(drop=True, inplace=True)

In [22]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5
1,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3
2,1030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,3,1030,506,H,LALakers,25,18,19,31,93,191,199,190,99
3,1031,701,V,Philadelphia,22,28,17,30,97,190,191,255,96.5,4,1031,702,H,Toronto,24,34,23,25,106,6.5,6.5,-305,2
4,1031,703,V,Washington,23,22,25,33,110,200,1.5,-125,1,5,1031,704,H,Indiana,28,20,22,33,119,1.5,203.5,105,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,608,503,V,LALakers,22,20,19,41,102,190,1.5,-128,5.5,1312,608,504,H,Boston,20,34,29,25,108,2,192,108,96.5
1312,610,505,V,Boston,20,17,25,19,81,195.5,195.5,425,97.5,1313,610,506,H,LALakers,20,23,17,27,87,9.5,9,-550,4
1313,612,507,V,Boston,14,26,31,26,97,192,191.5,275,3,1314,612,508,H,LALakers,35,23,15,18,91,8,7.5,-355,96.5
1314,615,509,V,Boston,22,30,18,28,98,190.5,193,255,97.5,1315,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4


In [23]:
#sanity check that every game only appears once
df['game_id']

0          1
1          2
2          3
3          4
4          5
        ... 
1311    1312
1312    1313
1313    1314
1314    1315
1315    1316
Name: game_id, Length: 1316, dtype: int64

In [24]:
df['Date'] = df['Date'].astype('str')
df['Date']

<ipython-input-24-0ac9846a9dad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = df['Date'].astype('str')


0       1030
1       1030
2       1030
3       1031
4       1031
        ... 
1311     608
1312     610
1313     612
1314     615
1315     617
Name: Date, Length: 1316, dtype: object

In [25]:
years = ['2007','2008','2008','2009','2009','2010','2010','2011','2011','2012','2012','2013','2013',
         '2014','2014','2015','2015','2016','2016','2017','2017','2018','2018','2019','2019','2020']

In [26]:
dates = []
for date in df['Date']:
    if int(date) > 1000:
        date = years[0]+date
        dates.append(date)
    else:
        date = years[1]+'0'+date
        dates.append(date)
df['Date'] = dates

<ipython-input-26-1a178c035ae4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = dates


In [27]:
df['Date']

0       20071030
1       20071030
2       20071030
3       20071031
4       20071031
          ...   
1311    20080608
1312    20080610
1313    20080612
1314    20080615
1315    20080617
Name: Date, Length: 1316, dtype: object

In [28]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2
0,20071030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5
1,20071030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3
2,20071030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,3,1030,506,H,LALakers,25,18,19,31,93,191,199,190,99
3,20071031,701,V,Philadelphia,22,28,17,30,97,190,191,255,96.5,4,1031,702,H,Toronto,24,34,23,25,106,6.5,6.5,-305,2
4,20071031,703,V,Washington,23,22,25,33,110,200,1.5,-125,1,5,1031,704,H,Indiana,28,20,22,33,119,1.5,203.5,105,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,20080608,503,V,LALakers,22,20,19,41,102,190,1.5,-128,5.5,1312,608,504,H,Boston,20,34,29,25,108,2,192,108,96.5
1312,20080610,505,V,Boston,20,17,25,19,81,195.5,195.5,425,97.5,1313,610,506,H,LALakers,20,23,17,27,87,9.5,9,-550,4
1313,20080612,507,V,Boston,14,26,31,26,97,192,191.5,275,3,1314,612,508,H,LALakers,35,23,15,18,91,8,7.5,-355,96.5
1314,20080615,509,V,Boston,22,30,18,28,98,190.5,193,255,97.5,1315,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4


In [29]:
df['Visitor'] = df['Team']
df['Home'] = df['Team_2']
df

<ipython-input-29-1744266c9047>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Visitor'] = df['Team']
<ipython-input-29-1744266c9047>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Home'] = df['Team_2']


,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,Visitor,Home
0,20071030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,Portland,SanAntonio
1,20071030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3,Utah,GoldenState
2,20071030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,3,1030,506,H,LALakers,25,18,19,31,93,191,199,190,99,Houston,LALakers
3,20071031,701,V,Philadelphia,22,28,17,30,97,190,191,255,96.5,4,1031,702,H,Toronto,24,34,23,25,106,6.5,6.5,-305,2,Philadelphia,Toronto
4,20071031,703,V,Washington,23,22,25,33,110,200,1.5,-125,1,5,1031,704,H,Indiana,28,20,22,33,119,1.5,203.5,105,105,Washington,Indiana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,20080608,503,V,LALakers,22,20,19,41,102,190,1.5,-128,5.5,1312,608,504,H,Boston,20,34,29,25,108,2,192,108,96.5,LALakers,Boston
1312,20080610,505,V,Boston,20,17,25,19,81,195.5,195.5,425,97.5,1313,610,506,H,LALakers,20,23,17,27,87,9.5,9,-550,4,Boston,LALakers
1313,20080612,507,V,Boston,14,26,31,26,97,192,191.5,275,3,1314,612,508,H,LALakers,35,23,15,18,91,8,7.5,-355,96.5,Boston,LALakers
1314,20080615,509,V,Boston,22,30,18,28,98,190.5,193,255,97.5,1315,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4,Boston,LALakers


In [36]:
df['Open'].replace('pk','0',inplace=True)
df['Close'].replace('pk','0',inplace=True)
df['Open_2'].replace('pk','0',inplace=True)
df['Close_2'].replace('pk','0',inplace=True)
df.replace('197.5u10','197.5',inplace=True)

/Users/jseemayer/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/series.py:4479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/Users/jseemayer/opt/anaconda3/envs/metis/lib/python3.8/site-packages/pandas/core/frame.py:4521: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [37]:
mask = df['Open'].values == 'pk'
df[mask]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,Visitor,Home


In [42]:
df['Open'] = df['Open'].astype('float')
df['Open_2'] = df['Open_2'].astype('float')
df['Close'] = df['Close'].astype('float')
df['Close_2'] = df['Close_2'].astype('float')

<ipython-input-42-d88ed8aac0b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Open'] = df['Open'].astype('float')
<ipython-input-42-d88ed8aac0b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Open_2'] = df['Open_2'].astype('float')
<ipython-input-42-d88ed8aac0b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

In [41]:
df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
df

<ipython-input-41-84833d6ad71f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])


,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,Visitor,Home,O/U_open
0,20071030,501,V,Portland,26,23,28,20,97,184.0,189.5,900,95,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,Portland,SanAntonio,184.0
1,20071030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2,1030,504,H,GoldenState,30,21,21,24,96,3.0,1,-120,3,Utah,GoldenState,214.5
2,20071030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,3,1030,506,H,LALakers,25,18,19,31,93,191.0,199,190,99,Houston,LALakers,191.0
3,20071031,701,V,Philadelphia,22,28,17,30,97,190.0,191,255,96.5,4,1031,702,H,Toronto,24,34,23,25,106,6.5,6.5,-305,2,Philadelphia,Toronto,190.0
4,20071031,703,V,Washington,23,22,25,33,110,200.0,1.5,-125,1,5,1031,704,H,Indiana,28,20,22,33,119,1.5,203.5,105,105,Washington,Indiana,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,20080608,503,V,LALakers,22,20,19,41,102,190.0,1.5,-128,5.5,1312,608,504,H,Boston,20,34,29,25,108,2.0,192,108,96.5,LALakers,Boston,190.0
1312,20080610,505,V,Boston,20,17,25,19,81,195.5,195.5,425,97.5,1313,610,506,H,LALakers,20,23,17,27,87,9.5,9,-550,4,Boston,LALakers,195.5
1313,20080612,507,V,Boston,14,26,31,26,97,192.0,191.5,275,3,1314,612,508,H,LALakers,35,23,15,18,91,8.0,7.5,-355,96.5,Boston,LALakers,192.0
1314,20080615,509,V,Boston,22,30,18,28,98,190.5,193,255,97.5,1315,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4,Boston,LALakers,190.5


In [43]:
df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
df

<ipython-input-43-f88eaef92b17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])


,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,game_id,Date_2,Rot_2,VH_2,Team_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,Visitor,Home,O/U_open,O/U_close
0,20071030,501,V,Portland,26,23,28,20,97,184.0,189.5,900,95,1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13.0,-1400,5,Portland,SanAntonio,184.0,189.5
1,20071030,503,V,Utah,28,34,24,31,117,214.5,212.0,100,105.5,2,1030,504,H,GoldenState,30,21,21,24,96,3.0,1.0,-120,3,Utah,GoldenState,214.5,212.0
2,20071030,505,V,Houston,16,27,27,25,95,2.5,5.0,-230,3,3,1030,506,H,LALakers,25,18,19,31,93,191.0,199.0,190,99,Houston,LALakers,191.0,199.0
3,20071031,701,V,Philadelphia,22,28,17,30,97,190.0,191.0,255,96.5,4,1031,702,H,Toronto,24,34,23,25,106,6.5,6.5,-305,2,Philadelphia,Toronto,190.0,191.0
4,20071031,703,V,Washington,23,22,25,33,110,200.0,1.5,-125,1,5,1031,704,H,Indiana,28,20,22,33,119,1.5,203.5,105,105,Washington,Indiana,200.0,203.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,20080608,503,V,LALakers,22,20,19,41,102,190.0,1.5,-128,5.5,1312,608,504,H,Boston,20,34,29,25,108,2.0,192.0,108,96.5,LALakers,Boston,190.0,192.0
1312,20080610,505,V,Boston,20,17,25,19,81,195.5,195.5,425,97.5,1313,610,506,H,LALakers,20,23,17,27,87,9.5,9.0,-550,4,Boston,LALakers,195.5,195.5
1313,20080612,507,V,Boston,14,26,31,26,97,192.0,191.5,275,3,1314,612,508,H,LALakers,35,23,15,18,91,8.0,7.5,-355,96.5,Boston,LALakers,192.0,191.5
1314,20080615,509,V,Boston,22,30,18,28,98,190.5,193.0,255,97.5,1315,615,510,H,LALakers,39,16,24,24,103,7.5,7.5,-305,4,Boston,LALakers,190.5,193.0


Sanity check that opening and closing lines operations worked successfully.
So long as numbers are around 200, we know we are ok.

In [46]:
df['O/U_open'].min()

172.0

In [47]:
df['O/U_close'].min()

171.5

In [48]:
df = df[['Date','Home','Visitor','O/U_open','O/U_close']]
df

,Date,Home,Visitor,O/U_open,O/U_close
0,20071030,SanAntonio,Portland,184.0,189.5
1,20071030,GoldenState,Utah,214.5,212.0
2,20071030,LALakers,Houston,191.0,199.0
3,20071031,Toronto,Philadelphia,190.0,191.0
4,20071031,Indiana,Washington,200.0,203.5
...,...,...,...,...,...
1311,20080608,Boston,LALakers,190.0,192.0
1312,20080610,LALakers,Boston,195.5,195.5
1313,20080612,LALakers,Boston,192.0,191.5
1314,20080615,LALakers,Boston,190.5,193.0


## Scratch Work Graveyard

In [ ]:
#tried self-merging on game_id, but this is only applicable if in different columns
#df1 = df1.merge(
#            right=df1[opp_pull_cols],
#            left_on=["game_id", "team"],
#            right_on=["game_id", "opp"],
#            suffixes=[None, "_opp"],
#        )

In [ ]:
#tried to use drop_duplicates method but can only keep first or last
#df.drop_duplicates(subset=['game_id'], keep='second')